<a href="https://colab.research.google.com/github/VGGatGitHub/Summer2021/blob/main/D-Wave_experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Experimenting how to use IBM DOcplex and D-Wave

# 0. Install libraries & clone repo
**You may need to restart the runtime after this cell**

In [ ]:
import sys

if hasattr(sys, 'real_prefix'):
    #we are in a virtual env.
    !pip install cplex && pip install docplex
else:
    !pip install --user cplex && pip install --user docplex

print()
!pip show cplex
print()
!pip show docplex

#it my be useful to retstart you runtime

In [ ]:
!pip install qiskit
!pip show qiskit

# !pip install qiskit-optimization
#!pip show qiskit-optimization

#it my be useful to retstart you runtime

In [ ]:
#it my be useful to retstart you runtime

# clone repo
!git clone https://github.com/VGGatGitHub/Summer2021.git

In [4]:
#!curl  -H "X-Auth-Token: YourAPIkey" -X DELETE https://cloud.dwavesys.com/sapi/problems/7211d196-cfc6-4630-84d0-f42ae5d55f0f

In [ ]:
#https://www.youtube.com/watch?v=bErs0dxC1aY&list=PLPvKnT7dgEsuhec_yW9oxcZ6gg9SfRFFV
#https://docs.ocean.dwavesys.com/en/latest/overview/install.html#installoceansoftware
#!python -m venv ocean
#!\Scripts\activate

!pip install dwave-ocean-sdk
!pip install dwave-qiskit-plugin

In [ ]:
!pip install requests --upgrade

In [ ]:
!dwave setup #API endpoint URL: https://cloud.dwavesys.com/sapi/

# 1. Restart your Runtime and then the Load Data

In [1]:
!pwd

/content


In [2]:
#next run the model.py in the folder introScenario
!pwd
#%cd ../intermediateScenario/
#%cd ./Summer2021/intermediateScenario
%cd ./Summer2021/introScenario/
#%ls


/content
/content/Summer2021/introScenario


In [3]:
import pandas as pd

# supply + costs
supply_file = './plants.csv'

# demand
demand_file = './customerDemand.csv'

# for now load example data (plants.csv and customerDemand.csv under introScenario folder)
s_df = pd.read_csv(supply_file)
d_df = pd.read_csv(demand_file)

In [4]:
d_df

,Product,Demand
0,handSanitizer,100
1,mask,120


In [5]:
s_df

,Plants,Cost,Capacity,Product
0,1,3,40,mask
1,2,2,30,mask
2,3,1,30,handSanitizer
3,4,3,100,handSanitizer
4,5,2,60,mask
5,6,1,45,mask


In [6]:
inputs={'customerDemand':d_df,'plants':s_df}

# 2. Solve the DO problem via DOcplex

In [7]:
#If you get an error you may have to restart the runtime
import cplex
import docplex.mp
from docplex.mp.model import Model


In [8]:
global output_lock
#with output_lock:

try:  
  if output_lock:
    print(output_lock)
except:
  output_lock=dict({'output_lock':{}})

try: 
  if outputs:
    print(outputs)
except:
  outputs=dict()


In [9]:
exec(open("./model.py").read())

* building wado model
model building done in 0.0939476490020752 secs
* running wado model
Version identifier: 20.1.0.0 | 2020-11-11 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
CPXPARAM_TimeLimit                               120
Found incumbent of value 0.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 20 rows and 12 columns.
All rows and columns eliminated.
Presolve time = 0.01 sec. (0.01 ticks)

Root node processing (before b&c):
  Real time             =    0.02 sec. (0.01 ticks)
Parallel b&c, 2 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.02 sec. (0.01 ticks)
Solve status: JobSolveStatus.OPTIMAL_SOLUTION
* model docplex_model1 solved with objective = -105680.000
*  KPI: total plants allocations                  = 112640.000
*  KPI: total Cost of plants over all allo

In [10]:
outputs


{'plantsAllocation':    plants  ...  plants minValueAllocationForAssignment
 0       1  ...                                       0
 1       2  ...                                       0
 2       3  ...                                       0
 3       4  ...                                       0
 4       5  ...                                       0
 5       6  ...                                       0
 
 [6 rows x 7 columns]}

In [11]:
outputs['plantsAllocation']

,plants,plants allocation decision,plants selection decision,plants Capacity,plants Product,plants Cost,plants minValueAllocationForAssignment
0,1,0.0,1.0,40,mask,3,0
1,2,15.0,1.0,30,mask,2,0
2,3,30.0,1.0,30,handSanitizer,1,0
3,4,70.0,1.0,100,handSanitizer,3,0
4,5,60.0,1.0,60,mask,2,0
5,6,45.0,1.0,45,mask,1,0


In [12]:
model

docplex.mp.Model['docplex_model1']

In [13]:
solution=model.solve()
print(solution)
solution

solution for: docplex_model1
objective: -105680
x2=15
x3=30
x4=70
x5=60
x6=45
x7=1
x8=1
x9=1
x10=1
x11=1
x12=1



docplex.mp.solution.SolveSolution(obj=-105680,values={x2:15,x3:30,x4:70,..

In [14]:
cost = solution.get_objective_value()
x = solution.get_all_values()
x = np.array(x)
print(cost,x)

-105680.0 [ 0. 15. 30. 70. 60. 45.  1.  1.  1.  1.  1.  1.]


#Simple qubo example for testing

In [45]:
#a simple example to test the code
from qiskit.optimization import QuadraticProgram
from qiskit.optimization.algorithms import CplexOptimizer
cplex = CplexOptimizer() #for binary and integer variables

qubo = QuadraticProgram()
qubo.binary_var('x')
qubo.binary_var('y')
qubo.binary_var('z')
qubo.minimize(linear=[1,-2,3], quadratic={('x', 'y'): 1, ('x', 'z'): -1, ('y', 'z'): 2})
print(qubo.export_as_lp_string())

solution=cplex.solve(qubo)
print(solution)

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Minimize
 obj: x - 2 y + 3 z + [ 2 x*y - 2 x*z + 4 y*z ]/2
Subject To

Bounds
 0 <= x <= 1
 0 <= y <= 1
 0 <= z <= 1

Binaries
 x y z
End

optimal function value: -2.0
optimal value: [0. 1. 0.]
status: SUCCESS


# 3. Try to Solve the DO problem using D-wave plugin

In [28]:
# take a look at https://quantum-computing.ibm.com/lab/docs/iql/optimization

#from qiskit_optimization.translators import from_docplex_mp

from qiskit.aqua.algorithms import QAOA, NumPyMinimumEigensolver
from qiskit.optimization.algorithms import CobylaOptimizer, MinimumEigenOptimizer
from qiskit.optimization.algorithms import CplexOptimizer
exact = MinimumEigenOptimizer(NumPyMinimumEigensolver()) # to solve QUBOs
cplex = CplexOptimizer()#for binary and integer variables
cobyla = CobylaOptimizer()#for continues variables


from qiskit.optimization import QuadraticProgram

# load from a Docplex model
mod = QuadraticProgram()
mod.from_docplex(model)


In [29]:
print(mod.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: docplex_model1

Minimize
 obj: - 464 x0 - 480 x1 - 496 x2 - 464 x3 - 480 x4 - 496 x5
Subject To
 _L_EXPR_1: x0 - 40 x6 <= 0
 _L_EXPR_2: x1 - 30 x7 <= 0
 _L_EXPR_3: x2 - 30 x8 <= 0
 _L_EXPR_4: x3 - 100 x9 <= 0
 _L_EXPR_5: x4 - 60 x10 <= 0
 _L_EXPR_6: x5 - 45 x11 <= 0
 _L_EXPR_7: x0 >= 0
 _L_EXPR_8: x1 >= 0
 _L_EXPR_9: x2 >= 0
 _L_EXPR_10: x3 >= 0
 _L_EXPR_11: x4 >= 0
 _L_EXPR_12: x5 >= 0
 _L_EXPR_13: x0 <= 40
 _L_EXPR_14: x1 <= 30
 _L_EXPR_15: x2 <= 30
 _L_EXPR_16: x3 <= 100
 _L_EXPR_17: x4 <= 60
 _L_EXPR_18: x5 <= 45
 _L_EXPR_19: x2 + x3 <= 100
 _L_EXPR_20: x0 + x1 + x4 + x5 <= 120

Bounds
 0 <= x6 <= 1
 0 <= x7 <= 1
 0 <= x8 <= 1
 0 <= x9 <= 1
 0 <= x10 <= 1
 0 <= x11 <= 1

Binaries
 x6 x7 x8 x9 x10 x11

Generals
 x0 x1 x2 x3 x4 x5
End



In [30]:
#take a look at the QUBO parameters 
print('constant:\t\t\t', mod.objective.constant)

print('linear dict:\t\t\t', mod.objective.linear.to_dict())
print('linear array:\t\t\t', mod.objective.linear.to_array())
print('linear array as sparse matrix:\n', mod.objective.linear.coefficients, '\n')

print('quadratic dict w/ index:\t', mod.objective.quadratic.to_dict())
print('quadratic dict w/ name:\t\t', mod.objective.quadratic.to_dict(use_name=True))
print('symmetric quadratic dict w/ name:\t', mod.objective.quadratic.to_dict(use_name=True, symmetric=True))
print('quadratic matrix:\n', mod.objective.quadratic.to_array(),'\n')
print('symmetric quadratic matrix:\n', mod.objective.quadratic.to_array(symmetric=True),'\n')
print('quadratic matrix as sparse matrix:\n', mod.objective.quadratic.coefficients)


constant:			 0.0
linear dict:			 {0: -464.0, 1: -480.0, 2: -496.0, 3: -464.0, 4: -480.0, 5: -496.0}
linear array:			 [-464. -480. -496. -464. -480. -496.    0.    0.    0.    0.    0.    0.]
linear array as sparse matrix:
   (0, 0)	-464.0
  (0, 1)	-480.0
  (0, 2)	-496.0
  (0, 3)	-464.0
  (0, 4)	-480.0
  (0, 5)	-496.0 

quadratic dict w/ index:	 {}
quadratic dict w/ name:		 {}
symmetric quadratic dict w/ name:	 {}
quadratic matrix:
 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]] 

symmetric quadratic matrix:
 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.

In [31]:
#compare the model and mod solutions 
print('\n',model.solve())

solution=cplex.solve(mod)
print(solution)



 solution for: docplex_model1
objective: -105680
x2=15
x3=30
x4=70
x5=60
x6=45
x7=1
x8=1
x9=1
x10=1
x11=1
x12=1

optimal function value: -105680.0
optimal value: [ 0. 15. 30. 70. 60. 45.  1.  1.  1.  1.  1.  1.]
status: SUCCESS


In [32]:
#convert the QP to all-binary QUBO 
from qiskit.optimization.converters import QuadraticProgramToQubo
conv = QuadraticProgramToQubo()
mod_Qubo = conv.convert(mod)

In [33]:
import dimod
import hybrid

import neal #sampler = neal.SimulatedAnnealingSampler()

from dwave.system import LeapHybridSampler
from dwave.system import DWaveSampler, EmbeddingComposite

# Extract qubo data

#comment the line below for testing with the simple qubo
#qubo=mod_Qubo #qp_eq_bin #mod_Qubo .to_array() 

Q = qubo.objective.quadratic.to_array()#.to_dict(use_name=True)
g = qubo.objective.linear.to_array()#.to_dict(use_name=True)
c = qubo.objective.constant

# Build BQM
bqm = dimod.BQM(g, Q, c, dimod.BINARY)

# Solve
#sampler = LeapHybridSampler()
sampler = neal.SimulatedAnnealingSampler()


In [35]:
from dwave.plugins.qiskit import DWaveMinimumEigensolver

# Solve using Qiskit's MinimumEigenOptimizer on D-Wave QPU as a minimum eigen solver
dwave_mes = DWaveMinimumEigensolver(sampler=sampler)
optimizer = MinimumEigenOptimizer(dwave_mes)


%time result = optimizer.solve(qubo) #use qubo for testing instead of mod_Qubo
print(result)
result.samples

CPU times: user 31.9 ms, sys: 1.9 ms, total: 33.8 ms
Wall time: 34.9 ms
optimal function value: -2.0
optimal value: [0. 1. 0.]
status: SUCCESS


[SolutionSample(x=array([0., 1., 0.]), fval=-2.0, probability=1.0, status=<OptimizationResultStatus.SUCCESS: 0>)]

In [ ]:
result.min_eigen_solver_result.sampleset.to_pandas_dataframe()

#try to modify the problem formulation 

In [77]:
mod.from_docplex(model)

#use the cost value as a quadratic part of the qubo

#quadratic=mod.objective.quadratic.to_array()
linear=s_df['Cost'].values
quadratic=np.kron(linear.reshape(1,-1),linear.reshape(-1,1))

#adding constrains
lagrange=10**5 #lagrange multiplier
linear1=np.array([1,1,0,0,1,1]) #120 masks demand constrain
linear2=np.array([0,0,1,1,0,0]) #100 sanitizers demand constrain
quadratic+=lagrange*np.kron(linear1.reshape(1,-1),linear1.reshape(-1,1))
quadratic+=lagrange*np.kron(linear2.reshape(1,-1),linear2.reshape(-1,1))

#linear=mod.objective.linear.to_array()
linear=2*120*linear1*lagrange+2*100*linear2*lagrange

#constant =mod.objective.constant
constant = lagrange*(120+100)**2

mod.minimize(constant=constant,  quadratic=quadratic)#,linear=-linear)
print(mod.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: docplex_model1

Minimize
 obj: [ 200018 x0^2 + 400024 x0*x1 + 12 x0*x2 + 36 x0*x3 + 400024 x0*x4
      + 400012 x0*x5 + 200008 x1^2 + 8 x1*x2 + 24 x1*x3 + 400016 x1*x4
      + 400008 x1*x5 + 200002 x2^2 + 400012 x2*x3 + 8 x2*x4 + 4 x2*x5
      + 200018 x3^2 + 24 x3*x4 + 12 x3*x5 + 200008 x4^2 + 400008 x4*x5
      + 200002 x5^2 ]/2 + 4840000000
Subject To
 _L_EXPR_1: x0 - 40 x6 <= 0
 _L_EXPR_2: x1 - 30 x7 <= 0
 _L_EXPR_3: x2 - 30 x8 <= 0
 _L_EXPR_4: x3 - 100 x9 <= 0
 _L_EXPR_5: x4 - 60 x10 <= 0
 _L_EXPR_6: x5 - 45 x11 <= 0
 _L_EXPR_7: x0 >= 0
 _L_EXPR_8: x1 >= 0
 _L_EXPR_9: x2 >= 0
 _L_EXPR_10: x3 >= 0
 _L_EXPR_11: x4 >= 0
 _L_EXPR_12: x5 >= 0
 _L_EXPR_13: x0 <= 40
 _L_EXPR_14: x1 <= 30
 _L_EXPR_15: x2 <= 30
 _L_EXPR_16: x3 <= 100
 _L_EXPR_17: x4 <= 60
 _L_EXPR_18: x5 <= 45
 _L_EXPR_19: x2 + x3 <= 100
 _L_EXPR_20: x0 + x1 + x4 + x5 <= 120

Bounds
 0 <= x6 <= 1
 0 <= x7 <= 1
 0 <= x8 <= 1
 0 <= x9 <= 1
 0 <= x

In [78]:
cplex.solve(mod)

optimal function value: 4840000000.0
optimal value: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
status: SUCCESS

In [79]:
#turn the demand inequalities into equalities
for eq_n in range(19,21):
  lc_name='_L_EXPR_'+str(eq_n)
  lin_c = mod.get_linear_constraint(lc_name)
  print('lin_c:',lc_name,lin_c.linear.to_dict(use_name=True), lin_c.sense, lin_c.rhs)
  mod.remove_linear_constraint(lc_name)
  mod.linear_constraint(lin_c.linear.to_dict(use_name=True), sense='==', rhs=lin_c.rhs, name=lc_name)

print(mod.export_as_lp_string())


lin_c: _L_EXPR_19 {'x2': 1.0, 'x3': 1.0} ConstraintSense.LE 100
lin_c: _L_EXPR_20 {'x0': 1.0, 'x1': 1.0, 'x4': 1.0, 'x5': 1.0} ConstraintSense.LE 120
\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: docplex_model1

Minimize
 obj: [ 200018 x0^2 + 400024 x0*x1 + 12 x0*x2 + 36 x0*x3 + 400024 x0*x4
      + 400012 x0*x5 + 200008 x1^2 + 8 x1*x2 + 24 x1*x3 + 400016 x1*x4
      + 400008 x1*x5 + 200002 x2^2 + 400012 x2*x3 + 8 x2*x4 + 4 x2*x5
      + 200018 x3^2 + 24 x3*x4 + 12 x3*x5 + 200008 x4^2 + 400008 x4*x5
      + 200002 x5^2 ]/2 + 4840000000
Subject To
 _L_EXPR_1: x0 - 40 x6 <= 0
 _L_EXPR_2: x1 - 30 x7 <= 0
 _L_EXPR_3: x2 - 30 x8 <= 0
 _L_EXPR_4: x3 - 100 x9 <= 0
 _L_EXPR_5: x4 - 60 x10 <= 0
 _L_EXPR_6: x5 - 45 x11 <= 0
 _L_EXPR_7: x0 >= 0
 _L_EXPR_8: x1 >= 0
 _L_EXPR_9: x2 >= 0
 _L_EXPR_10: x3 >= 0
 _L_EXPR_11: x4 >= 0
 _L_EXPR_12: x5 >= 0
 _L_EXPR_13: x0 <= 40
 _L_EXPR_14: x1 <= 30
 _L_EXPR_15: x2 <= 30
 _L_EXPR_16: x3 <= 100
 _L_EXPR_17: x4 <= 60
 _L_EXPR_

In [80]:
cplex.solve(mod)

optimal function value: 7280189225.0
optimal value: [-0. 15. 30. 70. 60. 45.  1.  1.  1.  1.  1.  1.]
status: SUCCESS

In [81]:
#remove the linear binary-integer relationship constrains  
for eq_n in range(1,7):
  lc_name='_L_EXPR_'+str(eq_n)
  mod.remove_linear_constraint(lc_name)

print(mod.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: docplex_model1

Minimize
 obj: [ 200018 x0^2 + 400024 x0*x1 + 12 x0*x2 + 36 x0*x3 + 400024 x0*x4
      + 400012 x0*x5 + 200008 x1^2 + 8 x1*x2 + 24 x1*x3 + 400016 x1*x4
      + 400008 x1*x5 + 200002 x2^2 + 400012 x2*x3 + 8 x2*x4 + 4 x2*x5
      + 200018 x3^2 + 24 x3*x4 + 12 x3*x5 + 200008 x4^2 + 400008 x4*x5
      + 200002 x5^2 ]/2 + 4840000000
Subject To
 _L_EXPR_7: x0 >= 0
 _L_EXPR_8: x1 >= 0
 _L_EXPR_9: x2 >= 0
 _L_EXPR_10: x3 >= 0
 _L_EXPR_11: x4 >= 0
 _L_EXPR_12: x5 >= 0
 _L_EXPR_13: x0 <= 40
 _L_EXPR_14: x1 <= 30
 _L_EXPR_15: x2 <= 30
 _L_EXPR_16: x3 <= 100
 _L_EXPR_17: x4 <= 60
 _L_EXPR_18: x5 <= 45
 _L_EXPR_19: x2 + x3 = 100
 _L_EXPR_20: x0 + x1 + x4 + x5 = 120

Bounds
 0 <= x6 <= 1
 0 <= x7 <= 1
 0 <= x8 <= 1
 0 <= x9 <= 1
 0 <= x10 <= 1
 0 <= x11 <= 1

Binaries
 x6 x7 x8 x9 x10 x11

Generals
 x0 x1 x2 x3 x4 x5
End



In [82]:
#remove the binary variables
bb2zero=dict({'x'+str(i):0 for i in range(6,12)})
sub = mod.substitute_variables(constants=bb2zero)
print(sub.export_as_lp_string())

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: docplex_model1

Minimize
 obj: [ 200018 x0^2 + 400024 x0*x1 + 12 x0*x2 + 36 x0*x3 + 400024 x0*x4
      + 400012 x0*x5 + 200008 x1^2 + 8 x1*x2 + 24 x1*x3 + 400016 x1*x4
      + 400008 x1*x5 + 200002 x2^2 + 400012 x2*x3 + 8 x2*x4 + 4 x2*x5
      + 200018 x3^2 + 24 x3*x4 + 12 x3*x5 + 200008 x4^2 + 400008 x4*x5
      + 200002 x5^2 ]/2 + 4840000000
Subject To
 _L_EXPR_7: x0 >= 0
 _L_EXPR_8: x1 >= 0
 _L_EXPR_9: x2 >= 0
 _L_EXPR_10: x3 >= 0
 _L_EXPR_11: x4 >= 0
 _L_EXPR_12: x5 >= 0
 _L_EXPR_13: x0 <= 40
 _L_EXPR_14: x1 <= 30
 _L_EXPR_15: x2 <= 30
 _L_EXPR_16: x3 <= 100
 _L_EXPR_17: x4 <= 60
 _L_EXPR_18: x5 <= 45
 _L_EXPR_19: x2 + x3 = 100
 _L_EXPR_20: x0 + x1 + x4 + x5 = 120

Bounds

Generals
 x0 x1 x2 x3 x4 x5
End



In [83]:
cplex.solve(sub)

optimal function value: 7280189225.0
optimal value: [-0. 15. 30. 70. 60. 45.]
status: SUCCESS

In [84]:
#remove the most variables but x2 and x3
if True :
  bb2zero=dict({'x'+str(i):0 for i in (list(range(0,2))+list(range(4,6)))})
  sub = sub.substitute_variables(constants=bb2zero)
print(sub.export_as_lp_string())

constraint _L_EXPR_20 is infeasible due to substitution


\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: docplex_model1

Minimize
 obj: [ 200002 x2^2 + 400012 x2*x3 + 200018 x3^2 ]/2 + 4840000000
Subject To
 _L_EXPR_9: x2 >= 0
 _L_EXPR_10: x3 >= 0
 _L_EXPR_15: x2 <= 30
 _L_EXPR_16: x3 <= 100
 _L_EXPR_19: x2 + x3 = 100

Bounds

Generals
 x2 x3
End



In [88]:
#convert the QP to all-binary QUBO 
from qiskit.optimization.converters import QuadraticProgramToQubo
conv = QuadraticProgramToQubo()
mod_Qubo = conv.convert(sub)
len(mod_Qubo._variables_index)

280

In [ ]:
print('constant:\t\t\t', mod_Qubo.objective.constant)
print('linear array as sparse matrix:\n', mod_Qubo.objective.linear.coefficients, '\n')
print('quadratic matrix as sparse matrix:\n', mod_Qubo.objective.quadratic.coefficients)

## Explore the QP to QUBO converter 

In [87]:
from qiskit.optimization.converters import InequalityToEquality
from qiskit.optimization.converters import IntegerToBinary
from qiskit.optimization.converters import LinearEqualityToPenalty

ineq2eq = InequalityToEquality()
qp_eq = ineq2eq.convert(mod)
#print(qp_eq.export_as_lp_string())

int2bin = IntegerToBinary()
qp_eq_bin = int2bin.convert(qp_eq)
#print(qp_eq_bin.export_as_lp_string())

lineq2penalty = LinearEqualityToPenalty()
mod_Qubo2 = lineq2penalty.convert(qp_eq_bin)
#print(qubo.export_as_lp_string())

len(mod_Qubo2._variables_index)

845

#try the D-wave again
https://quantumcomputing.stackexchange.com/questions/16622/from-qiskit-quadraticprogram-to-problem-model-accepted-by-dwave-ocean

In [108]:
import dimod
import hybrid

import neal #sampler = neal.SimulatedAnnealingSampler()

from dwave.system import LeapHybridSampler
from dwave.system import DWaveSampler, EmbeddingComposite

import neal


# Extract qubo data

qubo=mod_Qubo #qp_eq_bin #mod_Qubo .to_array() 

Q = qubo.objective.quadratic.to_array()#.to_dict(use_name=True)
g = qubo.objective.linear.to_array()#.to_dict(use_name=True)
c = qubo.objective.constant

# Build BQM
bqm = dimod.as_bqm(g, Q, c, dimod.BINARY)

# Solve
#sampler = LeapHybridSampler()
sampler = neal.SimulatedAnnealingSampler()


In [109]:
qubo=sub
cplex.solve(qubo)

optimal function value: 5840057600.0
optimal value: [30. 70.]
status: SUCCESS

In [ ]:
#solve it using D-wave

bqm.result = sampler.sample(bqm, label="mod_Qubo", num_reads=1024)

# Extract solution
result_dict = bqm.result.first.sample

print(bqm.result)

In [116]:
bqm.result.data_vectors

{'energy': array([-5.93550186e+71, -5.79746693e+71, -5.93550186e+71, ...,
        -5.93550186e+71, -6.21157171e+71, -6.34960664e+71]),
 'num_occurrences': array([1, 1, 1, ..., 1, 1, 1])}

In [ ]:
cplex.solve(mod_Qubo)

In [121]:
QuadraticProgramToQubo.interpret(mod_Qubo,bqm.result)

AttributeError: ignored

#More things to try

In [ ]:
from dwave_qbsolv import QBSolv
response = QBSolv().sample_qubo(Q)
print("samples=" + str(list(response.samples())))
print("energies=" + str(list(response.data_vectors['energy'])))

In [ ]:
bqm.result.data_vectors['energy']

array([-9.23177591e+73])

In [ ]:
op, offset = qubo.to_ising()
qp=QuadraticProgram()
qp.from_ising(op, offset, linear=True)
print(qp.export_as_lp_string())

In [ ]:
bqm1 = dimod.BinaryQuadraticModel.from_ising({}, {('a', 'b'): -1})
sampleset = dimod.SampleSet.from_samples_bqm({'a': -1, 'b': 1},bqm1)
print(bqm1)
print(sampleset)

BinaryQuadraticModel({a: 0.0, b: 0.0}, {('a', 'b'): -1}, 0, 'SPIN')
   a  b energy num_oc.
0 -1 +1    1.0       1
['SPIN', 1 rows, 1 samples, 2 variables]


In [ ]:
len(bqm.result.to_serializable()['variable_labels'])#['sample_data']['data']

845

In [ ]:
#sub.substitute_variables(bqm.result)